<a href="https://colab.research.google.com/github/pra08528/CODSOFT_ML_01/blob/main/code_movie_genre_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download -d hijest/genre-classification-dataset-imdb

Dataset URL: https://www.kaggle.com/datasets/hijest/genre-classification-dataset-imdb
License(s): other
 91% 38.0M/41.7M [00:00<00:00, 89.8MB/s]
100% 41.7M/41.7M [00:00<00:00, 91.4MB/s]


In [2]:
!unzip genre-classification-dataset-imdb.zip

Archive:  genre-classification-dataset-imdb.zip
  inflating: Genre Classification Dataset/description.txt  
  inflating: Genre Classification Dataset/test_data.txt  
  inflating: Genre Classification Dataset/test_data_solution.txt  
  inflating: Genre Classification Dataset/train_data.txt  


In [17]:
import pandas as pd

# Load the training data
train_file_path = 'Genre Classification Dataset/train_data.txt'
test_file_path = 'Genre Classification Dataset/test_data.txt'
test_solution_file_path = 'Genre Classification Dataset/test_data_solution.txt'

# Assuming each line is formatted as "plot_summary\tgenre"
def load_data(file_path):
    data = pd.read_csv(file_path, sep='\t', header=None, names=['plot', 'genre'])
    return data

train_data = load_data(train_file_path)
test_data = load_data(test_file_path)
test_solution_data = load_data(test_solution_file_path)

# Display a sample
print(train_data.head())


                                                plot  genre
0  1 ::: Oscar et la dame rose (2009) ::: drama :...    NaN
1  2 ::: Cupid (1997) ::: thriller ::: A brother ...    NaN
2  3 ::: Young, Wild and Wonderful (1980) ::: adu...    NaN
3  4 ::: The Secret Sin (1915) ::: drama ::: To h...    NaN
4  5 ::: The Unrecovered (2007) ::: drama ::: The...    NaN


In [18]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text

train_data['plot'] = train_data['plot'].apply(clean_text)
test_data['plot'] = test_data['plot'].apply(clean_text)


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(train_data['plot'])
X_test_tfidf = vectorizer.transform(test_data['plot'])


In [3]:
import pandas as pd

# Load the training data
train_data = pd.read_csv('Genre Classification Dataset/train_data.txt', delimiter='\t', header=None, names=['text', 'genre'])
train_data.head()


,text,genre
0,1 ::: Oscar et la dame rose (2009) ::: drama :...,NaN
1,2 ::: Cupid (1997) ::: thriller ::: A brother ...,NaN
2,"3 ::: Young, Wild and Wonderful (1980) ::: adu...",NaN
3,4 ::: The Secret Sin (1915) ::: drama ::: To h...,NaN
4,5 ::: The Unrecovered (2007) ::: drama ::: The...,NaN


In [25]:
print(train_data['genre'].isna().sum())
# Drop rows with missing values
train_data = train_data.dropna(subset=['genre'])
missing_data = train_data[train_data['genre'].isna()]
print(missing_data)

0
Empty DataFrame
Columns: [plot, genre]
Index: []


In [26]:
print(train_data['genre'].isna().sum())  # Should print 0 if all NaNs are handled


0


In [30]:
print("Number of samples in X_train_tfidf:", X_train_tfidf.shape[0])
print("Number of samples in train_data['genre']:", len(train_data['genre']))
# Re-check the number of rows in the original dataset and the split data

print("Training data size:", len(train_data))
print("Test data size:", len(test_data))


Number of samples in X_train_tfidf: 54214
Number of samples in train_data['genre']: 0
Training data size: 0
Test data size: 54200


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Reload and clean data
data = pd.read_csv('Genre Classification Dataset/train_data.txt', sep='\t', header=None, names=['plot', 'genre'])
data['plot'] = data['plot'].apply(clean_text)

# Split data
X = data['plot']
y = data['genre']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Recompute TF-IDF features
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("X_train_tfidf shape:", X_train_tfidf.shape)
print("y_train length:", len(y_train))


X_train_tfidf shape: (37949, 147716)
y_train length: 37949


In [32]:
# Remove rows with empty plot summaries
data = data.dropna(subset=['plot'])

# Check if any rows have missing genre labels
print(data[data['genre'].isna()])


                                                    plot  genre
0      1 oscar et la dame rose 2009 drama listening i...    NaN
1      2 cupid 1997 thriller a brother and sister wit...    NaN
2      3 young wild and wonderful 1980 adult as the b...    NaN
3      4 the secret sin 1915 drama to help their unem...    NaN
4      5 the unrecovered 2007 drama the film s title ...    NaN
...                                                  ...    ...
54209  54210 bonino 1953 comedy this short lived nbc ...    NaN
54210  54211 dead girls don t cry horror the next gen...    NaN
54211  54212 ronald goedemondt ze bestaan echt 2008 d...    NaN
54212  54213 make your own bed 1944 comedy walter and...    NaN
54213  54214 nature s fury storm of the century 2006 ...    NaN

[54214 rows x 2 columns]


In [33]:
# Check if there are any duplicate entries
print(data.duplicated().sum())


0


In [34]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Train the model
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_tfidf, y_train)
y_pred_svm = svm_classifier.predict(X_test_tfidf)

# Evaluate the model
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


ValueError: Input y contains NaN.

In [4]:
# Load the test data
test_data = pd.read_csv('Genre Classification Dataset/test_data.txt', delimiter='\t', header=None, names=['text'])
test_data.head()


,text
0,1 ::: Edgar's Lunch (1998) ::: L.R. Brane love...
1,"2 ::: La guerra de papá (1977) ::: Spain, Marc..."
2,3 ::: Off the Beaten Track (2010) ::: One year...
3,4 ::: Meu Amigo Hindu (2015) ::: His father ha...
4,5 ::: Er nu zhai (1955) ::: Before he was know...


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('text').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('index', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('text')):
  _plot_series(series, series_name, i)
  fig.legend(title='text', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_3['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_4['text'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_4, x='index', y='text', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [6]:
# Check for missing values in the target variable
print(train_data['genre'].isnull().sum())


54214


In [11]:
# Fill missing genres with a placeholder value (e.g., 'Unknown')
train_data['genre'].fillna('Unknown', inplace=True)
print(train_data['genre'].isnull().sum())  # Should print 0


0


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Ensure there are no empty or nearly empty rows
X = train_data['text']
y = train_data['genre']

# Remove empty rows
X = X[X.str.strip().astype(bool)]
y = y.loc[X.index]  # Align y with cleaned X

# Convert text data into TF-IDF features
tfidf = TfidfVectorizer(max_features=5000, min_df=1)  # Adjust parameters if needed
#X_tfidf = tfidf.fit_transform(X)

# Check the vocabulary size
#print(f"Vocabulary size: {len(tfidf.vocabulary_)}")

# Proceed if vocabulary is valid
if len(tfidf.vocabulary_) > 0:
    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

    # Initialize and train the Logistic Regression model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Predict on the validation set
    y_pred = model.predict(X_val)

    # Evaluate the model
    print("Accuracy:", accuracy_score(y_val, y_pred))
    print("Classification Report:\n", classification_report(y_val, y_pred))
else:
    print("Vocabulary is empty. Check your text data and preprocessing steps.")


AttributeError: 'TfidfVectorizer' object has no attribute 'vocabulary_'